# 1. Import

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import tensorflow
import keras
import ktrain
import pandas as pd
import io
import os.path
from ktrain import text
from sklearn.model_selection import train_test_split
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

using Keras version: 2.3.0


/opt/conda/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


/kaggle/input/reddit-comment-classification-comp-551/reddit_test.csv
/kaggle/input/reddit-comment-classification-comp-551/reddit_train.csv


In [2]:
!pip install ktrain
!pip install stellargraph

     |████████████████████████████████| 174kB 8.8MB/s eta 0:00:01
     |████████████████████████████████| 317kB 38.2MB/s eta 0:00:01
     |████████████████████████████████| 204kB 42.7MB/s eta 0:00:01
  Created wheel for ktrain: filename=ktrain-0.5.1-cp36-none-any.whl size=213555 sha256=f9dea90d47906fb380745848831481d27dfff1318ec17eecd22cfd6c7231f21a
  Stored in directory: /tmp/.cache/pip/wheels/a1/53/fb/6789e98ec03e43589ba5cf0f54a7aa26bdd80aeae31bc52786
  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=826b1a4a68ac6807cb294d85886b04d12fbf510910b33a241f540aad55bb7a5a
  Stored in directory: /tmp/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=8690f9d67664517fd9c157a82f45bc92a09cd1712a6a9156a4103fa6b02c6b5d
  Stored in directory: /tmp/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68


  Created wheel for keras-transformer: filename=keras_transformer-0.31.0-cp36-none-any.whl size=13385 sha256=d67f42bf8d02a7c97ccd830b98fe8b447621da0d0832ff57720f611bc356dd00
  Stored in directory: /tmp/.cache/pip/wheels/a3/c5/9a/5a5130240be614a7a6fa786765d7692ae97f82601e2161bb56
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7553 sha256=8806071442917a806655d27c0258512bbed855aa690fb8e45150342039be2265
  Stored in directory: /tmp/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=cd02ecf24d8e91a7001e579830bee3797052e991bb28770b2c3201af70132a6c
  Stored in directory: /tmp/.cache/pip/wheels/bb/df/3f/81b36f41b66e6a9cd69224c70a737de2bb6b2f7feb3272c25e
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.13.0-cp36-none-any.whl size=5209 sha256=586d1808117522b0cae6fe06360bda5f93bbb1a3a119e

# 2. Load Data

In [4]:
train = pd.read_csv('/kaggle/input/reddit-comment-classification-comp-551/reddit_train.csv',names=['id', 'comments', 'subreddits'],usecols = [1,2])
test = pd.read_csv('/kaggle/input/reddit-comment-classification-comp-551/reddit_test.csv',names=['id', 'comments'],usecols = [0,1])

In [5]:
train = train.iloc[1:]
train.head()

,comments,subreddits
1,"Honestly, Buffalo is the correct answer. I rem...",hockey
2,Ah yes way could have been :( remember when he...,nba
3,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends
4,He wouldn't have been a bad signing if we woul...,soccer
5,Easy. You use the piss and dry technique. Let ...,funny


In [6]:
test = test.iloc[1:]
test.head()

,id,comments
1,0,Trout and Bryant have both led the league in s...
2,1,&gt; Just like Estonians have good reasons to ...
3,2,Will Sol_Primeval sotp being oblivious?\n\nfin...
4,3,Moving Ostwald borders back to the pre 1967 bo...
5,4,"You have to take it out of the bag, Morty!"


In [7]:
index = dict((a,b) for b,a in enumerate(set(train['subreddits'])))
train['subreddits'] = train['subreddits'].apply(lambda x: index[x])
classnames = set(train['subreddits'])

In [13]:
print(index)

{'gameofthrones': 0, 'anime': 1, 'Music': 2, 'europe': 3, 'funny': 4, 'leagueoflegends': 5, 'nfl': 6, 'nba': 7, 'movies': 8, 'AskReddit': 9, 'wow': 10, 'canada': 11, 'trees': 12, 'GlobalOffensive': 13, 'conspiracy': 14, 'worldnews': 15, 'hockey': 16, 'soccer': 17, 'baseball': 18, 'Overwatch': 19}


In [12]:
print(train["subreddits"])

1        16
2         7
3         5
4        17
5         4
         ..
69996     3
69997     5
69998    11
69999     1
70000    13
Name: subreddits, Length: 70000, dtype: int64


In [11]:
print(classnames)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}


# 3. Vectorization

In [8]:
(x_train, y_train),(x_test, y_test),preproc = text.texts_from_df(train,preprocess_mode='bert',
                                            maxlen=512,
                                            val_pct=0.05,
                                            text_column='comments',
                                            label_columns=['subreddits'],
                                            max_features=62000,
                                            ngram_range=1,
                                            lang='en')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


# 4. Training & Evaluation

In [9]:
learner = ktrain.get_learner(text.text_classifier('bert', (x_train, y_train), preproc=preproc),
                            train_data=(x_train, y_train), 
                            val_data=(x_test, y_test), 
                            batch_size=6)

Is Multi-Label? False
maxlen is 512
done.


In [10]:
learner.fit(2e-5, 3)

Train on 63000 samples, validate on 7000 samples
Epoch 1/2
63000/63000 [==============================] - 5086s 81ms/step - loss: 1.5550 - accuracy: 0.5297 - val_loss: 1.3495 - val_accuracy: 0.5949
Epoch 2/2
63000/63000 [==============================] - 5055s 80ms/step - loss: 1.0820 - accuracy: 0.6694 - val_loss: 1.3391 - val_accuracy: 0.6110


# Log 
maxlen=512, val_pct=0.2, max_features=50000: - val_loss: 1.3555 - val_acc: 0.5930 
<br>
maxlen=512, val_pct=0.1, max_features=40000: - val_loss: 1.3518 - val_acc: 0.5881
<br>
maxlen=512, val_pct=0.1, max_features=50000: - val_loss: 1.3593 - val_acc: 0.5961
<br>
maxlen=400, val_pct=0.1, max_features=60000: - val_loss: 1.3436 - val_acc: 0.5974
<br>
maxlen=512, val_pct=0.1, max_features=50000: - val_loss: 1.3593 - val_acc: 0.5961
<br>
maxlen=512, val_pct=0.05, max_features=62000: - val_loss: 1.3389 - val_acc: 0.6072
<br>
maxlen=512, val_pct=0.2, max_features=500000: - val_loss: 1.3391 - val_acc: 0.6110


# 5. Prediction

In [14]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('predictor_final')

In [43]:
test_list = test.comments.values.tolist()

In [44]:
y_pred = predictor.predict(test_list,return_proba=True)

In [45]:
y_pred = np.asarray(y_pred)
positions= []
for ele in y_pred:
    pos = np.argmax(ele)
    positions.append(pos)

In [46]:
result = []
for p in positions:
    for name, num in index.items():
        if num == p:
            result.append(name)

In [47]:
final_res = pd.DataFrame({
                'Id': test.id,
                'Category':result
})

In [48]:
final_res.to_csv('predicted_result.csv',index=False)

In [49]:
from IPython.display import FileLink
FileLink(r'predicted_result.csv')

/kaggle/working/predicted_result.csv

In [50]:
FileLink(r'predictor_final')

/kaggle/working/predictor_final